<a href="https://colab.research.google.com/github/LeoFernanndes/portfolio/blob/master/GupyScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

pd.options.display.max_rows = 200

Teste de uma primeira requisição

In [2]:
req = requests.get("https://www.gupy.io/login/")

if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content

Requisição bem sucedida!


Antes de montar o loop, é importante entender a estrutura do html da página.
Vale começar dando uma olhada na documentação.

https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [3]:
# dando uma investigada na estrutura pra montar o loop
soup = BeautifulSoup(content, 'html.parser')
soup.find_all(name='tr')[1].find_all(name='td')[1].a.get('href')

'https://%20ghflynetwork.gupy.io/'

In [4]:
lista_empresas = []
lista_links = []

for tr in soup.find_all(name='tr'):  
  td_list = tr.find_all(name='td')
  
  if hasattr(td_list[1].a, 'href'): # escapar as linhas sem link    
    # empresa
    empresa = td_list[0].text
    lista_empresas.append(empresa)
    #link    
    link = td_list[1].a.get('href')
    lista_links.append(link)

df_empresas = pd.DataFrame({'Empresa': lista_empresas,
                            'Link': lista_links})

df_empresas

,Empresa,Link
0,Ghfly,https://%20ghflynetwork.gupy.io/
1,4All,https://4all.gupy.io/
2,A55,https://a55.gupy.io/
3,Banco ABC Brasil,https://abcbrasil.gupy.io/
4,AB InBev Chile,https://abinbevchile.gupy.io/
...,...,...
996,VerticalRH,https://verticalrh.gupy.io/
997,Vertigo,https://vertigo.gupy.io/
998,Vetta,https://vetta.gupy.io/
999,VEX LOGÍSTICA,https://vexlogistica.gupy.io/


In [5]:
lista_empresas = []
lista_vagas = []
lista_local = []
lista_remoto = []
lista_links = []

for i in range(df_empresas.shape[0]):

  nome_empresa = df_empresas.loc[i]['Empresa']
  link_empresa = df_empresas.loc[i]['Link']

  try:
    req_empresa = requests.get(link_empresa)
   
  except:
    continue
  
  if req_empresa.status_code != 200:
    continue
   

  content = req_empresa.content
  soup = BeautifulSoup(content, 'html.parser')

  for tr in soup.find_all(name='tr'):

    lista_empresas.append(nome_empresa)

    vaga = tr.find_all(name='td')[0].span.text.strip()
    lista_vagas.append(vaga)

    local = tr.find_all(name='td')[1].strong.text.replace('\n', '').strip()
    lista_local.append(local)

    remoto = 'remot' in tr.find_all(name='td')[1].strong.text.strip().lower()
    lista_remoto.append(remoto)

    link_relativo_vaga = tr.find_all(name='td')[0].a.get('href')
    link_vaga = link_empresa+link_relativo_vaga[1:]
    lista_links.append(link_vaga)

df_vagas = pd.DataFrame({
    'Empresa': lista_empresas,
    'Vaga': lista_vagas,
    'Local': lista_local,
    'Remoto': lista_remoto,
    'Link': lista_links
})

df_vagas

,Empresa,Vaga,Local,Remoto,Link
0,4All,Analista de Cybersecurity (APPSEC) PL/SR,Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/788992?jobBoardSourc...
1,4All,Analista de Marketing de Produto PL,Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/872683?jobBoardSourc...
2,4All,Analista de Marketing PL,Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/872852?jobBoardSourc...
3,4All,Analista de Testes PL,Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/752523?jobBoardSourc...
4,4All,Analista de UX PL/SR,Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/911835?jobBoardSourc...
...,...,...,...,...,...
20545,Viterra Bionergia,Mecanico Manutenção Agrícola - Guararapes,Guararapes,False,https://viterrabio.gupy.io/jobs/882834?jobBoar...
20546,Viterra Bionergia,Mecanico Manutenção Agricola -Junqueiropolis,Junqueirópolis/SP,False,https://viterrabio.gupy.io/jobs/882802?jobBoar...
20547,Viterra Bionergia,Operador ETA,Guararapes,False,https://viterrabio.gupy.io/jobs/940253?jobBoar...
20548,Viterra Bionergia,Operador Produção - Recepção Cana,Junqueirópolis/SP,False,https://viterrabio.gupy.io/jobs/940288?jobBoar...


In [6]:
from datetime import date

dia = str(date.today())

df_vagas.to_excel(f'vagas_gupy_{dia}.xlsx', index=False, encoding='utf-8')

In [7]:
df_vagas[(df_vagas['Vaga'].str.contains('data|dados', case=False)) & (df_vagas['Remoto'] == True)] # exemplo de filtragem

,Empresa,Vaga,Local,Remoto,Link
100,Accenture Brasil,Analista de Dados - 3627625,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/918121?jobBoard...
124,Accenture Brasil,Cientista de Dados - Big Data,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/918449?jobBoard...
128,Accenture Brasil,Consultor Big Data,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/918143?jobBoard...
142,Accenture Brasil,Data Engineer Consultant - 954956,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/917043?jobBoard...
143,Accenture Brasil,Data Engineering,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/817799?jobBoard...
...,...,...,...,...,...
20401,Will Bank,Data Scientist Sênior,São Paulo and Remote,True,https://vemserwill.gupy.io/jobs/900188?jobBoar...
20443,Vertigo,Analista de dados (Power BI ) - PJ,Remote Work,True,https://vertigo.gupy.io/jobs/915149?jobBoardSo...
20487,Vetta,Desenvolvedor(a) Especialista em Banco de Dados,Belo Horizonte/MG and Remote,True,https://vetta.gupy.io/jobs/814990?jobBoardSour...
20508,Vetta,Engenheiro(a) de Dados - Sênior,Betim/MG and Remote,True,https://vetta.gupy.io/jobs/790304?jobBoardSour...


In [8]:
df_vagas[(df_vagas['Vaga'].str.contains('data|dados', case=False)) & (df_vagas['Remoto'] == True)] # exemplo de filtragem

,Empresa,Vaga,Local,Remoto,Link
100,Accenture Brasil,Analista de Dados - 3627625,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/918121?jobBoard...
124,Accenture Brasil,Cientista de Dados - Big Data,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/918449?jobBoard...
128,Accenture Brasil,Consultor Big Data,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/918143?jobBoard...
142,Accenture Brasil,Data Engineer Consultant - 954956,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/917043?jobBoard...
143,Accenture Brasil,Data Engineering,São Paulo/SP and Remote,True,https://accenture.gupy.io/jobs/817799?jobBoard...
...,...,...,...,...,...
20401,Will Bank,Data Scientist Sênior,São Paulo and Remote,True,https://vemserwill.gupy.io/jobs/900188?jobBoar...
20443,Vertigo,Analista de dados (Power BI ) - PJ,Remote Work,True,https://vertigo.gupy.io/jobs/915149?jobBoardSo...
20487,Vetta,Desenvolvedor(a) Especialista em Banco de Dados,Belo Horizonte/MG and Remote,True,https://vetta.gupy.io/jobs/814990?jobBoardSour...
20508,Vetta,Engenheiro(a) de Dados - Sênior,Betim/MG and Remote,True,https://vetta.gupy.io/jobs/790304?jobBoardSour...


In [9]:
df_vagas[(df_vagas['Vaga'].str.contains('python|java', case=False)) & (df_vagas['Remoto'] == True)] # exemplo de filtragem

,Empresa,Vaga,Local,Remoto,Link
5,4All,Dev Back-end Java SR,Porto Alegre/RS and Remote,True,https://4all.gupy.io/jobs/653141?jobBoardSourc...
19,A55,Desenvolvedor Java Sr.,São Paulo and Remote,True,https://a55.gupy.io/jobs/801468?jobBoardSource...
21,A55,Desenvolvedor Python Júnior Remoto - a55,Manaus and Remote,True,https://a55.gupy.io/jobs/849140?jobBoardSource...
22,A55,Desenvolvedor Python Pleno Remoto - a55,Manaus and Remote,True,https://a55.gupy.io/jobs/849082?jobBoardSource...
23,A55,Desenvolvedor Python Sênior Remoto - a55,São Paulo and Remote,True,https://a55.gupy.io/jobs/744457?jobBoardSource...
...,...,...,...,...,...
20490,Vetta,Desenvolvedor(a) Full Stack - Java Pleno,Ipatinga/MG and Remote,True,https://vetta.gupy.io/jobs/893444?jobBoardSour...
20491,Vetta,Desenvolvedor(a) Full Stack - Java Pleno,Ipatinga/MG and Remote,True,https://vetta.gupy.io/jobs/838071?jobBoardSour...
20493,Vetta,Desenvolvedor(a) Java - Pleno,Remote Work,True,https://vetta.gupy.io/jobs/899724?jobBoardSour...
20494,Vetta,Desenvolvedor(a) Java - Sênior #1,Betim/MG and Remote,True,https://vetta.gupy.io/jobs/834606?jobBoardSour...


In [10]:
df_vagas[(df_vagas['Vaga'].str.contains('jr|junior', case=False))] # exemplo de filtragem

,Empresa,Vaga,Local,Remoto,Link
25,Banco ABC Brasil,Analista de Compliance JR,São Paulo/SP,False,https://abcbrasil.gupy.io/jobs/865691?jobBoard...
260,Agenda Edu,Desenvolvedor(a) React Junior,Fortaleza/CE and Remote,True,https://agendaedu.gupy.io/jobs/877688?jobBoard...
262,Agenda Edu,Desenvolvedor(a) Ruby on Rails Junior,Fortaleza/CE and Remote,True,https://agendaedu.gupy.io/jobs/894716?jobBoard...
303,Agrotis,Analista de Faturamento Junior,Curitiba/PR,False,https://agrotis.gupy.io/jobs/915045?jobBoardSo...
312,Agrotools,Analista Financeiro Jr/Pleno - Presencial e/ou...,São Paulo/SP and Remote,True,https://agrotools.gupy.io/jobs/896182?jobBoard...
...,...,...,...,...,...
20057,Valecard,Assistente Técnico Jr,Uberlândia,False,https://valecard.gupy.io/jobs/650526?jobBoardS...
20432,VerticalRH,Analista de Recrutamento e Seleção - Jr ou Pleno,São Paulo/SP,False,https://verticalrh.gupy.io/jobs/940102?jobBoar...
20434,VerticalRH,Analista de RH Jr - Generalista,São Paulo/SP,False,https://verticalrh.gupy.io/jobs/940194?jobBoar...
20435,VerticalRH,Analista de T&D - Jr ou Pleno,São Paulo/SP,False,https://verticalrh.gupy.io/jobs/940326?jobBoar...
